# Optimize Initial Conditions
## Yabox

In [1]:
from scipy.optimize import brute
import numpy as np
from datetime import datetime,timedelta
import pandas as pd
from yabox import DE

In [2]:
# Load solver
import LearnerICRayNoLoad as L 

In [3]:
def load_confirmed(state, start_date):
    dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
    df = pd.read_csv('./data/confirmados.csv',delimiter=',',parse_dates=True, date_parser=dateparse)
    y=[]
    x=[]
    for i in range(0,len(df.date)):
        y.append(df[state].values[i])
        x.append(df.date.values[i])
    df2=pd.DataFrame(data=y,index=x,columns=[""])
    df2=df2[start_date:]
    del x,y
    return df2

def load_dead(state, start_date):
    dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
    df = pd.read_csv('./data/mortes.csv',delimiter=',',parse_dates=True, date_parser=dateparse)
    y=[]
    x=[]
    for i in range(0,len(df.date)):
        y.append(df[state].values[i])
        x.append(df.date.values[i])
    df2=pd.DataFrame(data=y,index=x,columns=[""])
    df2=df2[start_date:]
    return df2

In [4]:
# Initial parameters
dfparam = pd.read_csv("data/param.csv")
dfparam

,state,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC
0,SP,2020-03-15,200,8000000.0,0,0,800,300,250,100,0.15,0.55,0.01
1,ES,2020-04-01,200,475000.0,0,0,50,250,50,100,0.15,0.65,0.01
2,MG,2020-04-01,200,1100000.0,0,0,200,250,40,100,0.15,0.55,0.01
3,RJ,2020-03-20,200,700000.0,0,0,800,250,50,100,0.15,0.50,0.01
4,CE,2020-03-20,200,800000.0,0,0,800,250,50,100,0.15,0.50,0.01
5,PE,2020-03-20,200,700000.0,0,0,800,250,100,100,0.15,0.50,0.01
6,AM,2020-03-20,200,700000.0,0,0,800,250,100,100,0.15,0.50,0.01


In [5]:
df = pd.read_csv("data/dados_total_estados.csv")
df

,index,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,popEst,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,0,2020-06-06,SE,TOTAL,state,8972,208,74,True,2298696.0,28.0,390.30824,0.0232
1,1,2020-06-05,SE,TOTAL,state,8577,198,73,False,2298696.0,28.0,373.12459,0.0231
2,2,2020-06-04,SE,TOTAL,state,8200,186,72,False,2298696.0,28.0,356.72399,0.0227
3,3,2020-06-03,SE,TOTAL,state,7989,180,71,False,2298696.0,28.0,347.54487,0.0225
4,4,2020-06-02,SE,TOTAL,state,7555,172,70,False,2298696.0,28.0,328.66460,0.0228
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2244,158210,2020-03-11,BA,TOTAL,state,3,0,5,False,14873064.0,29.0,0.02017,0.0000
2245,158211,2020-03-10,BA,TOTAL,state,2,0,4,False,14873064.0,29.0,0.01345,0.0000
2246,158212,2020-03-09,BA,TOTAL,state,2,0,3,False,14873064.0,29.0,0.01345,0.0000
2247,158213,2020-03-07,BA,TOTAL,state,2,0,2,False,14873064.0,29.0,0.01345,0.0000


In [6]:
from scipy.integrate import odeint
import os
import sys
import io
import ray
import gc
import tensorflow as tf

def create_f(state,e0,a0,r0,d0,date,startNCases, ratio, predict_range, version):
                
    def fobjective(point):
        
        dead=  load_dead(state,date)
        data = load_confirmed(state,date)*(1-ratio)-dead
        
        cleanRecovered=False
        s0, deltaDate, i0, weigthCases, weigthRecov = point
        end_date=datetime.strptime(date, "%Y-%m-%d") + timedelta(days=+int(deltaDate))
                    
        f=L.Learner.remote(state, end_date.strftime("%Y-%m-%d"), predict_range,\
                           s0, e0, a0, i0, r0, d0, startNCases, ratio, weigthCases, weigthRecov, \
                           cleanRecovered, version, data, dead, savedata=True)
        result = f.train.remote() 
        result = ray.get(result) 

        del end_date,cleanRecovered, data, dead         

        gc.collect()

        return result
    return fobjective

In [7]:
@ray.remote(memory=50 * 1024 * 1024, max_calls=1)
def opt(state,s0,i0,e0,a0,r0,d0,date,startNCases, ratio, predict_range, version):

    bounds=[(0.8*s0,1.2*s0),(-10,10),(0.8*i0,1.2*i0),\
              (0.3,0.7),(.001,.05)]
    maxiterations=300
    f=create_f(state,e0,a0,r0,d0,date,startNCases, ratio, predict_range, version)
    de = DE(f, bounds, maxiters=maxiterations)
    for step in de.geniterator():
        try:
            idx = step.best_idx
            norm_vector = step.population[idx]
            best_params = de.denormalize([norm_vector])
            del norm_vector, idx
        except:
            print("error in function evaluation")
    p=best_params[0]
    del f, bounds, data, dead
    
    return p

In [8]:
states=['SP','ES','MG','RJ','PE','CE','AM']
# states=['MG','RJ','PE','CE','AM']
allStates=True
version="100"
ray.shutdown()
GB=1024*1024*1024
MB=1024*1024
ray.init(object_store_memory=500*MB,memory=2*GB,lru_evict=True,\
         driver_object_store_memory=50*MB,num_cpus=16)
gc.enable()

optimal=[]
if allStates:
    for state in states:
        #remove previous history file
        strFile='./results/history_'+state+version+'.csv'
        if os.path.isfile(strFile):
            os.remove(strFile)
        query = dfparam.query('state == "{}"'.format(state)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        date,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
        optimal.append(opt.remote(state,s0,i0,e0,a0,r0,d0,date,startNCases, ratio, predict_range, version))        
else:
    state = "ES" 
    #remove previous history file
    strFile='./results/history_'+state+version+'.csv'
    if os.path.isfile(strFile):
        os.remove(strFile)
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    start_date,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
    optimal.append(opt.remote(state,s0,i0,e0,a0,r0,d0,date,startNCases, ratio, predict_range, version))        



2020-06-12 14:34:28,467	INFO resource_spec.py:212 -- Starting Ray with 1.95 GiB memory available for workers and up to 0.49 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-06-12 14:34:28,909	INFO services.py:1170 -- View the Ray dashboard at localhost:8266


In [ ]:
optimal=ray.get(optimal)


(pid=91345) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=91345)   warnings.warn(warning_msg, ODEintWarning)
(pid=raylet) E0612 14:37:31.344858 90096 node_manager.cc:3537] Failed to send get core worker stats request: IOError: 2: HandleServiceClosed
(pid=93976) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=93976)   warnings.warn(warning_msg, ODEintWarning)
(pid=94790) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=94790)   warnings.warn(warning_msg, ODEintWarning)
(pid=96267) /home/a

(pid=132429) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=132429)   warnings.warn(warning_msg, ODEintWarning)
(pid=135095) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=135095)   warnings.warn(warning_msg, ODEintWarning)
(pid=135941) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=135941)   warnings.warn(warning_msg, ODEintWarning)
(pid=136492) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong

In [ ]:
for i in range(0,len(states)):    

    state=states[i]
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    startdate,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
    Date = datetime.strptime(startdate, "%Y-%m-%d")
    end_date = Date + timedelta(days=+int(optimal[i][1]))
    dateStr=end_date.strftime("%Y-%m-%d")

    j = query['index'].values[0]
    dfparam.at[j, "s0"] = optimal[i][0]
    dfparam.at[j, "start-date"] = dateStr
    dfparam.at[j, "i0"] = optimal[i][2]
    dfparam.at[j, "WCASES"] = optimal[i][3]
    dfparam.at[j, "WREC"] = optimal[i][4]

    dfparam.to_csv("data/param_optimized_Yabox.csv", sep=",", index=False)
    dfparam
    